In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"

using CSV, DataFrames, Muon

In [4]:
clonedf = CSV.read(datadir("exp_raw", "BB673", "cloneList.biobank.relax.csv"),DataFrame; header=1)

,msk_id,clones,species
,String15,Int64,String
1,MSK.1.13,1,[Clostridium] aldenense
2,MSK.1.17,1,[Clostridium] aldenense
3,MSK.2.26,1,[Clostridium] clostridioforme
4,MSK.2.78,1,[Clostridium] clostridioforme
5,MSK.2.94,1,[Clostridium] clostridioforme
6,MSK.2.98,1,[Clostridium] clostridioforme
7,MSK.2.73,1,[Clostridium] clostridioforme
8,MSK.2.90,1,[Clostridium] clostridioforme
9,MSK.2.80,1,[Clostridium] clostridioforme


In [11]:
allunique(clonedf.msk_id)

true

In [5]:
biobank = readh5mu(datadir("exp_pro", "BB673", "BB673.h5mu"))

MuData object 673 ✕ 11296
└ metabolites_foldchange
  AnnData object 673 ✕ 38
└ metabolites_millimolar
  AnnData object 673 ✕ 10
└ oggs
  AnnData object 673 ✕ 11248

In [16]:
UID = first.(split.(biobank.obs_names, "_"))
basedf = DataFrame(
    :msk_id => UID,
    :donor_id => getindex.(split.(UID, "."), 2),
);

In [17]:
allunique(basedf.msk_id)

true

In [22]:
biobank.obs[:, :msk_id] .= UID;

In [50]:
mergedf = leftjoin(basedf, clonedf, on=:msk_id);
# mergedf = leftjoin(mergedf, biobank.obs, on=:msk_id, makeunique=true);

In [51]:
mergedf[findall(ismissing.(mergedf.clones)), :]

,msk_id,donor_id,clones,species
,SubStrin…,SubStrin…,Int64?,String?
1,MSK.13.52,13,missing,missing
2,DFI.1.109,1,missing,missing
3,DFI.5.19,5,missing,missing
4,DFI.1.159,1,missing,missing
5,DFI.5.15,5,missing,missing
6,DFI.1.221,1,missing,missing
7,DFI.5.34,5,missing,missing
8,DFI.1.219,1,missing,missing
9,DFI.1.64,1,missing,missing


In [26]:
names(mergedf)

11-element Vector{String}:
 "msk_id"
 "donor_id"
 "clones"
 "species"
 "phylum"
 "class"
 "order"
 "family"
 "genus"
 "species_1"
 "donor"

In [47]:
clonesvsisolates = mergedf |>
    df -> filter(:clones => x -> .!ismissing.(x), df) |>
    df -> transform(df, :species_1 => :species) |>
    df -> groupby(df, [:species, :donor]) |>
    df -> combine(df, :clones => (x -> length(unique(x))) => :num_clones, 
                      nrow => :num_isolates) |>
    df -> sort(df, [:species, :num_clones], rev=true)

,species,donor,num_clones,num_isolates
,String?,String?,Int64,Int64
1,unclassified,19,2,3
2,unclassified,16,2,3
3,unclassified,10,1,1
4,unclassified,15,1,1
5,unclassified,21,1,1
6,unclassified,22,1,1
7,unclassified,13,1,1
8,unclassified,20,1,1
9,unclassified,9,1,1


In [46]:
rdir = projectdir("_research", "BB673_clonalitymetrics") |> mkpath

"/Users/ben/projects/Doran_etal_2022/_research/BB673_clonalitymetrics"

In [53]:
CSV.write(joinpath(rdir, "BB673_clonesvsisolates.tsv"), clonesvsisolates; delim="\t")

"/Users/ben/projects/Doran_etal_2022/_research/BB673_clonalitymetrics/BB673_clonesvsisolates.tsv"

In [54]:
CSV.write(joinpath(rdir, "BB673_missingcloneinfo.tsv"), mergedf[findall(ismissing.(mergedf.clones)), :]; delim="\t")

"/Users/ben/projects/Doran_etal_2022/_research/BB673_clonalitymetrics/BB673_missingcloneinfo.tsv"